In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/amex-default-prediction/sample_submission.csv
/kaggle/input/amex-default-prediction/train_data.csv
/kaggle/input/amex-default-prediction/test_data.csv
/kaggle/input/amex-default-prediction/train_labels.csv
/kaggle/input/amexfeather/test_data_f32.ftr
/kaggle/input/amexfeather/train_data.ftr
/kaggle/input/amexfeather/train_data_f32.ftr
/kaggle/input/amexfeather/test_data.ftr


Despite the risk of sampling a biased sample, it's still pretty useful early on in EDA to use subsamples of the data in order to avoid choking the computer. I have chosen to load the first 10k data points of the dataset.

In [2]:
sample_submission = pd.read_csv('/kaggle/input/amex-default-prediction/sample_submission.csv', header=0, nrows=10000)
train_data = pd.read_csv('/kaggle/input/amex-default-prediction/train_data.csv', header=0, nrows=10000)
test_data = pd.read_csv('/kaggle/input/amex-default-prediction/test_data.csv', header=0, nrows=10000)
train_labels = pd.read_csv('/kaggle/input/amex-default-prediction/train_labels.csv', header=0, nrows=10000)

In [3]:
train_data.head()

,customer_ID,S_2,P_2,D_39,B_1,B_2,R_1,S_3,D_41,B_3,...,D_136,D_137,D_138,D_139,D_140,D_141,D_142,D_143,D_144,D_145
0,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2017-03-09,0.938469,0.001733,0.008724,1.006838,0.009228,0.124035,0.008771,0.004709,...,NaN,NaN,NaN,0.002427,0.003706,0.003818,NaN,0.000569,0.000610,0.002674
1,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2017-04-07,0.936665,0.005775,0.004923,1.000653,0.006151,0.126750,0.000798,0.002714,...,NaN,NaN,NaN,0.003954,0.003167,0.005032,NaN,0.009576,0.005492,0.009217
2,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2017-05-28,0.954180,0.091505,0.021655,1.009672,0.006815,0.123977,0.007598,0.009423,...,NaN,NaN,NaN,0.003269,0.007329,0.000427,NaN,0.003429,0.006986,0.002603
3,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2017-06-13,0.960384,0.002455,0.013683,1.002700,0.001373,0.117169,0.000685,0.005531,...,NaN,NaN,NaN,0.006117,0.004516,0.003200,NaN,0.008419,0.006527,0.009600
4,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2017-07-16,0.947248,0.002483,0.015193,1.000727,0.007605,0.117325,0.004653,0.009312,...,NaN,NaN,NaN,0.003671,0.004946,0.008889,NaN,0.001670,0.008126,0.009827


In [4]:
train_data.describe()

,P_2,D_39,B_1,B_2,R_1,S_3,D_41,B_3,D_42,D_43,...,D_136,D_137,D_138,D_139,D_140,D_141,D_142,D_143,D_144,D_145
count,9936.000000,1.000000e+04,10000.000000,10000.000000,10000.000000,8248.000000,1.000000e+04,10000.000000,1355.000000,6913.000000,...,372.000000,372.000000,372.000000,9.847000e+03,9.937000e+03,9.847000e+03,1572.000000,9847.000000,9.937000e+03,9.847000e+03
mean,0.650498,1.570308e-01,0.126147,0.617122,0.074035,0.227838,6.473782e-02,0.135379,0.173210,0.153766,...,0.229368,0.015669,0.218863,1.646316e-01,2.286722e-02,1.505506e-01,0.370263,0.162935,4.636371e-02,5.784571e-02
std,0.252416,2.758269e-01,0.212428,0.403145,0.219977,0.196746,1.969007e-01,0.238910,0.188531,0.203790,...,0.205791,0.103492,0.308344,3.662716e-01,1.323064e-01,3.347093e-01,0.249771,0.364742,1.741972e-01,2.269450e-01
min,-0.256921,8.701630e-07,-0.141469,0.000034,0.000003,-0.151649,6.841272e-07,0.000003,0.000697,0.000010,...,0.000045,0.000003,0.000024,5.934267e-07,5.821456e-07,2.562315e-07,-0.008804,0.000002,7.773110e-07,1.577788e-07
25%,0.471264,4.573938e-03,0.009126,0.091656,0.002863,0.128615,2.998132e-03,0.005326,0.046385,0.040649,...,0.008208,0.002268,0.004214,2.940035e-03,2.627201e-03,3.001612e-03,0.150429,0.002939,2.674433e-03,2.966105e-03
50%,0.690195,9.296279e-03,0.032948,0.814048,0.005726,0.163527,5.866382e-03,0.010243,0.133838,0.081796,...,0.253035,0.005048,0.008086,5.969675e-03,5.191741e-03,5.909631e-03,0.352169,0.005944,5.323101e-03,5.931741e-03
75%,0.866060,2.380151e-01,0.122675,1.002262,0.008528,0.260673,8.820872e-03,0.169040,0.236997,0.180020,...,0.258497,0.007452,0.503998,8.946506e-03,7.664635e-03,8.828591e-03,0.559880,0.008907,8.104901e-03,8.878259e-03
max,1.009926,4.268383e+00,1.323411,1.009999,2.259283,1.803549,2.592768e+00,1.258546,2.556439,2.320434,...,1.005479,1.009248,1.507652,1.009997e+00,1.009970e+00,1.112668e+00,1.185992,1.010000,1.342362e+00,4.282032e+00
